In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")



# NOTE: Whichever package you want mention here.
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'


In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

In [3]:
#connection details
url = "jdbc:mysql://127.0.0.1:3306/test?useSSL=false&allowPublicKeyRetrieval=true"
driver = "com.mysql.jdbc.Driver"
user = "bigdata"
password = "Bigdata@123"


In [4]:
df_customers1 =  spark.read\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", url)\
    .option("user", user)\
    .option("password", password)\
    .option("dbtable", "project.customers")\
    .option("numPartitions", 2)\
    .load()


In [5]:
df_customers1.show(3)

+--------------------+------------------------+--------------------+---------------+
|         customer_id|customer_zip_code_prefix|       customer_city| customer_state|
+--------------------+------------------------+--------------------+---------------+
|
|
|
+--------------------+------------------------+--------------------+---------------+
only showing top 3 rows



In [7]:
df_customers=df_customers1.exceptAll(df_customers1.limit(1))

df_customers.show(3)

+--------------------+------------------------+-------------+--------------+
|         customer_id|customer_zip_code_prefix|customer_city|customer_state|
+--------------------+------------------------+-------------+--------------+
|
|
|
+--------------------+------------------------+-------------+--------------+
only showing top 3 rows



In [23]:
#from pyspark.sql.functions import trim
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def trim(s):
    return s.strip()

udf_trim = udf(trim, StringType())

df_c = df_customers.withColumn("customer_state", udf_trim("customer_state"))




In [24]:
df_payments1 =  spark.read\
    .format("jdbc")\
    .option("driver", driver)\
    .option("url", url)\
    .option("user", user)\
    .option("password", password)\
    .option("dbtable", "project.payments")\
    .load()


In [25]:
df_payments=df_payments1.exceptAll(df_payments1.limit(1))
df_payments.show(3)

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|8e47a04b25d7a7075...|                 1| credit_card|                   1|        37.61|
|8e47a04b25d7a7075...|                 1| credit_card|                   1|        37.61|
|6b58f3523e45f9838...|                 1| credit_card|                   1|        44.23|
+--------------------+------------------+------------+--------------------+-------------+
only showing top 3 rows



In [26]:
df_orders1 = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("file:///home/talentum/project/orders.csv") \




In [27]:
df_orders=df_orders1.exceptAll(df_orders1.limit(1))

In [28]:
df_orders.show(3)



+--------------------+--------------------+------------+------------------------+-----------------+-------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_timestamp|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-----------------+-------------------------+-----------------------------+
|01835ef41c5ac66dd...|f64ee7b44a9c2170a...|   delivered|        11-05-2017 21:36| 11-05-2017 21:45|         23-05-2017 22:13|             01-06-2017 00:00|
|d5211af24819d319c...|a5931be4783e1a1df...|   delivered|        03-08-2018 16:29| 03-08-2018 16:45|         09-08-2018 15:09|             05-09-2018 00:00|
|d85553b6a0d18ab73...|6ee9dd7a3a539c1ae...|   delivered|        09-06-2018 12:33| 09-06-2018 12:58|         20-06-2018 18:52|             11-07-2018 00:00|
+--------------------+--------------------+------------+--------

In [29]:
df_order_items = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("file:///home/talentum/project/order_items.csv") \



In [30]:
df_order_items.show(3,truncate=False)

+--------------------------------+-------------+--------------------------------+--------------------------------+-----+----------------+
|order_id                        |order_item_id|product_id                      |seller_id                       |price|shipping_charges|
+--------------------------------+-------------+--------------------------------+--------------------------------+-----+----------------+
|00010242fe8c5a6d1ba2dd792cb16214|1            |4244733e06e7ecb4970a6e2683c13e61|48436dade18ac8b2bce089ec2a041202|58.9 |13.29           |
|00018f77f2f0320c557190d7a144bdd3|1            |e5f2d52b802189ee658865ca93d83a8f|dd7ddc04e1b6c2c614352b383efe2d36|239.9|19.93           |
|000229ec398224ef6ca0657da4fc703e|1            |c777355d18b72b67abbeef9df44fd0fd|5b51032eddd242adc84c38acab88f23d|199.0|17.87           |
+--------------------------------+-------------+--------------------------------+--------------------------------+-----+----------------+
only showing top 3 rows



In [31]:
df_products = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("file:///home/talentum/project/products.csv") 



In [32]:
df_products.show(3)

+--------------------+---------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|            perfumery|             225|               16|               10|              14|
|3aa071139cb16b67c...|                  art|            1000|               30|               18|              20|
|96bd76ec8810374ed...|       sports_leisure|             154|               18|                9|              15|
+--------------------+---------------------+----------------+-----------------+-----------------+----------------+
only showing top 3 rows



In [36]:
# Register DataFrames as temporary SQL tables
# df_c.createOrReplaceTempView("customers")
# df_payments.createOrReplaceTempView("payments")
# df_products.createOrReplaceTempView("products")
# df_order_items.createOrReplaceTempView("order_items")
# df_orders.createOrReplaceTempView("orders")
from pyspark.sql.functions import col

retail_inner_join = df_c.alias("t1").join(df_orders.alias("t2"), col("t1.customer_id") == col("t2.customer_id")) \
    .join(df_payments.alias("t3"), col("t2.order_id") == col("t3.order_id")) \
    .join(df_order_items.alias("t4"), col("t3.order_id") == col("t4.order_id")) \
    .join(df_products.alias("t5"), col("t4.product_id") == col("t5.product_id")) \
    .select(
        col("t1.customer_id"),
        col("t1.customer_city"),
        col("t1.customer_state"),
        col("t2.order_id"),
        col("t2.order_status"),
        col("t2.order_approved_at"),
        col("t2.order_delivered_timestamp"),
        col("t3.payment_sequential"),
        col("t3.payment_type"),
        col("t3.payment_value"),
        col("t4.order_item_id"),
        col("t4.product_id"),
        col("t4.seller_id"),
        col("t5.product_category_name"))
    



In [37]:
retail_inner_join.show(3,truncate=True)

+--------------------+-------------+--------------+--------------------+------------+-----------------+-------------------------+------------------+------------+-------------+-------------+--------------------+--------------------+---------------------+
|         customer_id|customer_city|customer_state|            order_id|order_status|order_approved_at|order_delivered_timestamp|payment_sequential|payment_type|payment_value|order_item_id|          product_id|           seller_id|product_category_name|
+--------------------+-------------+--------------+--------------------+------------+-----------------+-------------------------+------------------+------------+-------------+-------------+--------------------+--------------------+---------------------+
|e8d87ee946600f775...|     mesquita|            RJ|014405982914c2cde...|   delivered| 26-07-2017 17:50|         31-07-2017 15:53|                 1| credit_card|        78.43|            2|e95ee6822b66ac605...|a17f621c590ea0fab...|       

In [38]:

dictionary={'SP': 'São Paulo',
 'SC': 'Santa Catarina',
 'MG': 'Minas Gerais',
 'PR': 'Paraná',
 'RJ': 'Rio de Janeiro',
 'RS': 'Rio Grande do Sul',
 'PA': 'Pará',
 'GO': 'Goiás',
 'ES': 'Espírito Santo',
 'BA': 'Bahia',
 'MA': 'Maranhão',
 'MS': 'Mato Grosso do Sul',
 'CE': 'Ceará',
 'DF': 'Distrito Federal',
 'RN': 'Rio Grande do Norte',
 'PE': 'Pernambuco',
 'MT': 'Mato Grosso',
 'AM': 'Amazonas',
 'AP': 'Amapá',
 'AL': 'Alagoas',
 'RO': 'Rondônia',
 'PB': 'Paraíba',
 'TO': 'Tocantins',
 'PI': 'Piauí',
 'AC': 'Acre',
 'SE': 'Sergipe',
 'RR': 'Roraima'}




In [39]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def translate(dictionary):
    
    return udf(lambda col: dictionary.get(col),StringType())
  

  
# Create a new column by calling the function to map the values
df_state=retail_inner_join.withColumn("state",
              translate(dictionary)("customer_state"))




In [40]:
df_state.select("customer_city", "customer_state", "state").show(3)


+-------------+--------------+--------------+
|customer_city|customer_state|         state|
+-------------+--------------+--------------+
|     mesquita|            RJ|Rio de Janeiro|
|     mesquita|            RJ|Rio de Janeiro|
|     mesquita|            RJ|Rio de Janeiro|
+-------------+--------------+--------------+
only showing top 3 rows



In [41]:
print(df_state.columns)

['customer_id', 'customer_city', 'customer_state', 'order_id', 'order_status', 'order_approved_at', 'order_delivered_timestamp', 'payment_sequential', 'payment_type', 'payment_value', 'order_item_id', 'product_id', 'seller_id', 'product_category_name', 'state']


In [42]:
dictionary_product={
  "perfumery": "Health and Beauty",
  "art": "Art",
  "sports_leisure": "Sports and Leisure",
  "baby": "Baby",
  "housewares": "Housewares",
  "musical_instruments": "Musical Instruments",
  "cool_stuff": "Cool Stuff",
  "furniture_decor": "Furniture",
  "home_appliances": "Home Appliances",
  "toys": "Toys",
  "bed_bath_table": "Homewares",
  "construction_tools_safety": "Construction Tools",
  "computers_accessories": "Electronics",
  "health_beauty": "Health and Beauty",
  "luggage_accessories": "Travel",
  "garden_tools": "Garden",
  "office_furniture": "Furniture",
  "auto": "Automotive",
  "electronics": "Electronics",
  "fashion_shoes": "Fashion",
  "telephony": "Telephony",
  "stationery": "Stationery",
  "fashion_bags_accessories": "Fashion",
  "computers": "Electronics",
  "home_construction": "Home Improvement",
  "watches_gifts": "Gifts",
  "construction_tools_construction": "Construction Tools",
  "pet_shop": "Pets",
  "small_appliances": "Home Appliances",
  "agro_industry_and_commerce": "Business",
  "NA": "Others",
  "furniture_living_room": "Furniture",
  "signaling_and_security": "Security",
  "air_conditioning": "Home Appliances",
  "consoles_games": "Entertainment",
  "books_general_interest": "Books",
  "costruction_tools_tools": "Construction Tools",
  "fashion_underwear_beach": "Fashion",
  "fashion_male_clothing": "Fashion",
  "kitchen_dining_laundry_garden_furniture": "Furniture",
  "industry_commerce_and_business": "Business",
  "fixed_telephony": "Telephony",
  "construction_tools_lights": "Construction Tools",
  "books_technical": "Books",
  "home_appliances_2": "Home Appliances",
  "party_supplies": "Others",
  "drinks": "Food and Drink",
  "market_place": "Others",
  "la_cuisine": "Home Appliances",
  "costruction_tools_garden": "Garden",
  "fashio_female_clothing": "Fashion",
  "home_confort": "Home Improvement",
  "audio": "Electronics",
  "food_drink": "Food and Drink",
  "music": "Entertainment",
  "food": "Food and Drink",
  "tablets_printing_image": "Electronics",
  "books_imported": "Books",
  "small_appliances_home_oven_and_coffee": "Home Appliances",
  "fashion_sport": "Sports and Leisure",
  "christmas_supplies": "Others",
  "fashion_childrens_clothes": "Fashion",
  "dvds_blu_ray": "Entertainment",
  "arts_and_craftmanship": "Arts and Crafts",
  "furniture_bedroom": "Furniture",
  "cine_photo": "Entertainment",
  "diapers_and_hygiene": "Baby",
  "flowers": "Home and Garden",
  "home_comfort_2": "Home Improvement",
  "security_and_services": "Security",
  "furniture_mattress_and_upholstery": "Furniture"
}

# def find_unique_values(dict):
#   values = set(dict.values())
#   return values

# print(find_unique_values(dictionary_product))

In [43]:
# Create a new column by calling the function to map the values
df_category=df_state.withColumn("category",
              translate(dictionary_product)("product_category_name"))

In [45]:
df_category.select("product_category_name", "category", "state").show(3)

+---------------------+--------+--------------+
|product_category_name|category|         state|
+---------------------+--------+--------------+
|                 toys|    Toys|Rio de Janeiro|
|                 toys|    Toys|Rio de Janeiro|
|                 toys|    Toys|Rio de Janeiro|
+---------------------+--------+--------------+
only showing top 3 rows



In [46]:
df_category.columns


['customer_id',
 'customer_city',
 'customer_state',
 'order_id',
 'order_status',
 'order_approved_at',
 'order_delivered_timestamp',
 'payment_sequential',
 'payment_type',
 'payment_value',
 'order_item_id',
 'product_id',
 'seller_id',
 'product_category_name',
 'state',
 'category']

In [47]:
#df.drop(["order_approved_at", "order_delivered_timestamp"])
df_final=df_category.drop("customer_state", "order_approved_at", "order_delivered_timestamp","product_category_name")
df_final.columns

['customer_id',
 'customer_city',
 'order_id',
 'order_status',
 'payment_sequential',
 'payment_type',
 'payment_value',
 'order_item_id',
 'product_id',
 'seller_id',
 'state',
 'category']

In [48]:
 # Saving modes
df_final.write.mode('overwrite').option("path","/home/talentum/retail_data").saveAsTable("retail")